In [2]:
 import cv2
import numpy as np

cap = cv2.VideoCapture(0)

prev_frame = None
motion_detected = False
timer = 0
detect_motion = True

while True:
    ret, in_frame = cap.read()
    if not ret:
        break

    frame = cv2.GaussianBlur(in_frame, (33, 33), 0)
    gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    if prev_frame is not None:
        frame_diff = cv2.absdiff(prev_frame, gray_frame)

        _, thresh = cv2.threshold(frame_diff, 25, 255, cv2.THRESH_BINARY)

        contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

        if len(contours) > 0:
            motion_detected = True
        else:
            motion_detected = False

    prev_frame = gray_frame

    motion_map = np.zeros_like(frame)
    motion_map[:] = (0, 255, 0)
    if motion_detected:
        motion_map[thresh > 0] = (0, 0, 255)

    combined_frame = np.hstack((in_frame, motion_map))
    cv2.imshow('Motion Detection', combined_frame)

    key = cv2.waitKey(20) & 0xff
    if key == 27:
        break

cv2.destroyWindow('Motion Detection')
cap.release()

